In [59]:
import pandas as pd 
import numpy as np
import os
import json
import zipfile
import re
from scipy.spatial import cKDTree

pd.options.display.max_columns = 160


In [60]:
# General Functions 

# Function to clean strings and convert to lowercase
def transform_place_name(place_name):
    # if place_name is str: 
    place_name = place_name.lower()
    place_name = place_name.strip()
    place_name = place_name.replace('-', '')
        
        # Split and swap the parts if there's a comma
    if ',' in place_name:
        parts = place_name.split(',')
        if len(parts) > 1:
            # Strip extra spaces, rearrange parts, and replace spaces with hyphens
            new_place_name = (parts[1].strip() + ' ' + parts[0].strip()).replace(' ', '')
            return new_place_name

    # Replace spaces with hyphens for names without a comma
    place_name = re.sub(r"[^\w\s]", '', place_name)
    place_name = place_name.replace(' ', '')
    return place_name.replace('-', '')

# Define a function to adjust the year in the 'Datum' column
def adjust_year(date):
    try:
        return date.replace(year=date.year - 1)
    except ValueError:
        if date.month == 2 and date.day == 29:
            return pd.NaT  # Use pandas Not a Time to flag this date for removal
        else:
            raise

In [61]:
# Set the folder path
folder_path = 'data_traffic_jams'

# Get the list of files in the folder
files = os.listdir(folder_path)

# Initialize an empty list to store the dataframes
dfs = []

# Define the search pattern once, escaping parentheses
search_pattern = 'Spitsfile \(geen oorzaak gemeld\)|File buiten spits \(geen oorzaak gemeld\)'

# Define valid years set for checking file year
valid_years = {'2019', '2020', '2021', '2022', '2023', '2024'}

# Iterate over the files
for file in files:
    year = file[:4]
    # Check if the file is from 2018 onwards
    if year in valid_years:
        # Read each file as a dataframe
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, delimiter=';')
        
        # Filter rows where 'Oorzaak_1' contains any of the specified values
        filtered_df = df[df['OorzaakGronddetail'].str.contains(search_pattern, case=False, regex=True)]
        
        # Append the filtered dataframe to the list
        dfs.append(filtered_df)

# Concatenate the dataframes into one big dataframe
df = pd.concat(dfs, ignore_index=True)

# Efficiently convert and combine date and time columns
df['DatumTijdFileBegin'] = pd.to_datetime(df['DatumFileBegin'] + ' ' + df['TijdFileBegin'])
df['DatumTijdFileEind'] = pd.to_datetime(df['DatumFileEind'] + ' ' + df['TijdFileEind'])
df['DatumFileBeginInt'] = df['DatumTijdFileBegin'].dt.strftime('%Y%m%d').astype(int)
df['DatumFileEindInt'] = df['DatumTijdFileEind'].dt.strftime('%Y%m%d').astype(int)


# Perform string replacements in bulk instead of looping through each column
for suffix in ['-Noord', '-Oost', '-Zuid', '-West']:
    df['KopWegvakNaar'] = df['KopWegvakNaar'].str.replace(suffix, '', regex=False)
    df['KopWegvakVan'] = df['KopWegvakVan'].str.replace(suffix, '', regex=False)
    df['TrajVan'] = df['TrajVan'].str.replace(suffix, '', regex=False)
    df['TrajNaar'] = df['TrajNaar'].str.replace(suffix, '', regex=False)


# Fill NaN values and convert columns to strings efficiently
columns_to_convert = ['KopWegvakVan', 'KopWegvakNaar']
df[columns_to_convert] = df[columns_to_convert].fillna('').astype(str)

# Apply transformation to place names
df['KopWegvakVan'] = df['KopWegvakVan'].apply(transform_place_name)
df['KopWegvakNaar'] = df['KopWegvakNaar'].apply(transform_place_name)
df['TrajNaar'] = df['TrajNaar'].apply(transform_place_name)
df['TrajVan'] = df['TrajVan'].apply(transform_place_name)

# Correctly handle decimal separators and convert to float
for column in ['FileZwaarte', 'GemLengte', 'FileDuur']:
    df[column] = df[column].str.replace(',', '.').astype(float)

# Define a dictionary with replacements
replacements = {
    'mariënheem': 'marienheem',
    'belgië': 'breda',
    'mariănheem': 'marienheem',
    'portzélande': 'ouddorp',
    'schöninghsdorf': 'arnhem',
    'belgiă': 'breda',
    'portzălande': 'ouddorp',
    'portzalande': 'ouddorp',
    'MariĂ«nheem': 'zwolle', 
    'nederland': 'roosendaal'  # Assuming 'MariĂ«nheem' is equivalent to 'zwolle'
}

# Apply the replacements to both 'KopWegvakNaar' and 'KopWegvakVan' columns
df['KopWegvakNaar'] = df['KopWegvakNaar'].replace(replacements, regex=True)
df['KopWegvakVan'] = df['KopWegvakVan'].replace(replacements, regex=True)

# Convert hecotmeter columns to float
df['HectometerKop'] = df['HectometerKop'].str.replace(',', '.').astype(float)
df['HectometerStaart'] = df['HectometerStaart'].str.replace(',', '.').astype(float)


In [63]:
# Fuel data

# Load and preprocess the fuel data
fuel_df = pd.read_csv('/Users/floris/Desktop/DSS/Thesis/data_fuel/Observations.csv', delimiter=';')

fuel_df.drop(["StringValue", "ValueAttribute"], axis=1, inplace=True)

# Mapping measure codes to fuel types
measure_to_fuel = {
    'A047220': 'Gasoline',
    'A047219': 'Diesel',
    'A047221': 'LPG'
}
fuel_df['Measure'] = fuel_df['Measure'].map(measure_to_fuel).fillna(fuel_df['Measure'])

# Pivot the DataFrame so each fuel type is a column with the date as rows
fuel_df = fuel_df.pivot_table(index='Perioden', columns='Measure', values='Value', aggfunc='first')

# Merge the data
df = df.merge(fuel_df, left_on='DatumFileBeginInt', right_index=True, how='left')

df['Gasoline'] = df['Gasoline'].str.replace(',', '.').astype(float)
df['Diesel'] = df['Diesel'].str.replace(',', '.').astype(float)
df['LPG'] = df['LPG'].str.replace(',', '.').astype(float)

In [64]:
# Read the newplacenames data
newplacenames_df = pd.read_csv('data_area/nieuw_placenames.csv', sep=';', encoding='latin1')
newplacenames_df['Nieuwe'] = newplacenames_df['Nieuwe'].apply(transform_place_name)
newplacenames_df['UniqueKopWegvakNaar'] = newplacenames_df['UniqueKopWegvakNaar'].apply(transform_place_name)
replacement_dict = newplacenames_df.set_index('UniqueKopWegvakNaar')['Nieuwe'].to_dict()

# Create two new columns for KopWegvakVan and KopWegvakNaar with their new values
df['KopWegvakVan_New'] = df['KopWegvakVan'].map(replacement_dict)
df['KopWegvakNaar_New'] = df['KopWegvakNaar'].map(replacement_dict)

df['KopWegvakVan_New'] = df['KopWegvakVan_New'].astype('string')
df['KopWegvakNaar_New'] = df['KopWegvakNaar_New'].astype('string')


In [65]:
# Read the '4pp.csv' dataset
area_df = pd.read_csv('/Users/floris/Desktop/DSS/Thesis/data_area/4pp.csv', delimiter=',')

# Convert 'latitude' and 'longitude' to numeric, coercing non-numeric values to NaN
area_df['latitude'] = pd.to_numeric(area_df['latitude'], errors='coerce')
area_df['longitude'] = pd.to_numeric(area_df['longitude'], errors='coerce')

# Calculate average longitude and latitude for each distinct place name, ignoring NaNs
area_df = area_df.groupby('woonplaats').agg({'latitude': 'mean', 'longitude': 'mean', 'provincie': 'first'}).reset_index()

# Apply transformations to the 'PlaceName' column
area_df['woonplaats'] = area_df['woonplaats'].astype('string')
area_df['woonplaats'] = area_df['woonplaats'].apply(transform_place_name)

# Merge the 'provincie' column from 'area_df' with the main DataFrame 'df' based on 'KopWegvakVan_New' and 'KopWegvakNaar_New'
df = pd.merge(df, area_df, left_on='KopWegvakVan_New', right_on='woonplaats', how='left')
df = pd.merge(df, area_df, left_on='KopWegvakNaar_New', right_on='woonplaats', how='left')

# Drop the additional 'PlaceName' columns
# df.drop(['PlaceName_x', 'PlaceName_y', 'provincie_y'], axis=1, inplace=True)

In [67]:
station_df= pd.read_csv('data_weather/stations.csv')

# Remove NaN values from critical columns to avoid errors
df = df.dropna(subset=['latitude_x', 'longitude_x'])
station_df = station_df.dropna(subset=['latitude', 'longitude'])

# Extract unique coordinates from merged_df
unique_coords = df[['latitude_x', 'longitude_x']].drop_duplicates().reset_index(drop=True)

# Create a k-d tree with station coordinates
tree = cKDTree(station_df[['latitude', 'longitude']].values)

# Query the tree for the two nearest neighbors of each unique coordinate
distances, indices = tree.query(unique_coords.values, k=2)  # k=2 for the two nearest stations

# Extract the closest station coordinates using the indices from the k-d tree query
closest_stations = station_df.iloc[indices[:, 0]].reset_index().rename(columns={'index': 'original_index', 'station_code': 'closest_station_code'})
second_closest_stations = station_df.iloc[indices[:, 1]].reset_index().rename(columns={'index': 'original_index', 'station_code': 'second_closest_station_code'})

# Include the station codes from the closest and second closest stations to map back
unique_coords['closest_station_code'] = closest_stations['closest_station_code']
unique_coords['second_closest_station_code'] = second_closest_stations['second_closest_station_code']

# Merge this information back onto the original merged_df to map each entry to its closest and second closest station
# Ensure latitude_x and longitude_x uniquely identify rows in merged_df,
# if not, additional merging criteria may be needed.
merged_df = pd.merge(df, unique_coords, on=['latitude_x', 'longitude_x'], how='left')


In [69]:
folder_path = '/Users/floris/Desktop/DSS/Thesis/data_weather'

# Get the list of file names in the folder
file_names = os.listdir(folder_path)

# Initialize an empty list to store the dataframes
dfs = []

# Iterate over the file names
for file_name in file_names:
    # Check if the file is a ZIP file
    if file_name.endswith('.zip'):
        # Construct the file path
        zip_path = os.path.join(folder_path, file_name)
        
        # Open the ZIP file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # List all the contained files to ensure we only process JSON files
            contained_files = zip_ref.namelist()
            for contained_file in contained_files:
                if contained_file.endswith('.json'):
                    # Extract JSON file content
                    with zip_ref.open(contained_file) as file:
                        data = json.load(file)
                    
                    # Convert the JSON data to a dataframe
                    df = pd.DataFrame(data)
                    
                    # Append the dataframe to the list
                    dfs.append(df)

# Concatenate the dataframes
if dfs:
    weather_df = pd.concat(dfs, ignore_index=True)
else:
    print("No JSON files found in the ZIP archives.")

weather_df = weather_df.drop_duplicates()
weather_df['hour'] -=1 

# weather_df.to_csv('weather_data.csv', index=False)
# weather_df = pd.read_csv('weather_data.csv')


/var/folders/6d/z_ycnlvn2kb2n3lr05wsg2cm0000gn/T/ipykernel_5695/1821816312.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat(dfs, ignore_index=True)


In [70]:
df = merged_df.copy()
# Change the NaN values in the Boolean columns to 0 
columns_to_fill = ['M', 'R', 'S', 'O', 'Y']
weather_df[columns_to_fill] = weather_df[columns_to_fill].fillna(0)

# Example: Ensuring 'station_index' and 'station_code' are both integers
df['closest_station_code'] = df['closest_station_code'].astype(int)
weather_df['station_code'] = weather_df['station_code'].astype(int)

# Similarly, make sure that 'HourOfDay' and 'hour' are of the same type, as well as 'DatumFileBegin' and 'date'
df['DatumTijdFileBegin'] = pd.to_datetime(df['DatumTijdFileBegin'], format="%Y-%m-%d %H:%M:%S")
df['HourOfDay'] = df['DatumTijdFileBegin'].dt.hour 
df['HourOfDay'] = df['HourOfDay'].astype(int)
weather_df['hour'] = weather_df['hour'].astype(int)

# Similarly, make sure that 'HourOfDay' and 'hour' are of the same type, as well as 'DatumFileBegin' and 'date'
df['DatumFileBeginInt'] = df['DatumFileBeginInt'].astype(int)
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df['date'] = weather_df['date'].dt.strftime('%Y%m%d')
weather_df['date'] = weather_df['date'].astype(int)


In [71]:
# First merge with the closest station
df = pd.merge(
    df, weather_df,
    how='left',
    left_on=['HourOfDay', 'DatumFileBeginInt', 'closest_station_code'],
    right_on=['hour', 'date', 'station_code'],
    suffixes=('', '_closest')
)

# Second merge with the second closest station
result_second_df = pd.merge(
    df, weather_df,
    how='left',
    left_on=['HourOfDay', 'DatumFileBeginInt', 'second_closest_station_code'],
    right_on=['hour', 'date', 'station_code'],
    suffixes=('', '_second_closest')
)

columns_to_check =['DD', 'FH', 'FF', 'FX', 'T', 'SQ', 'Q', 'DR', 'RH', 'P', 'VV', 'N', 'U']

# Replace NaN values from the first merge with values from the second merge
for column in columns_to_check:
    df[column] = df[column].fillna(result_second_df[column + '_second_closest'])


In [72]:
directory = 'data_public_transport'  # Specify your directory path
files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Initialize an empty list to store DataFrames
df_list = []

# Loop through each file and append its DataFrame to the list
for file in files:
    file_path = os.path.join(directory, file)  # Create the full path to the file
    dfs = pd.read_csv(file_path)  # Read the CSV file
    df_list.append(dfs)  # Append the DataFrame to the list

# Concatenate all DataFrames in the list into a single DataFrame
pt_df = pd.concat(df_list, ignore_index=True)

# Ensure the Datum column exists
if 'Datum' not in pt_df.columns or pt_df['Datum'].isnull().all():
    pt_df['Datum'] = pd.NaT

# Filter to get indices where Datum is NaN
na_indices = pt_df[pt_df['Datum'].isna()].index

# Step 2: Construct Datum only for NaN entries
for idx in na_indices:
    # Create a date string from 'Dag', 'Maand', 'Jaar'
    day = pt_df.loc[idx, 'Dag']
    month = pt_df.loc[idx, 'Maand']
    year = pt_df.loc[idx, 'Jaar']
    if pd.notna(day) and pd.notna(month) and pd.notna(year):
        date_str = f"{int(day)}-{int(month)}-{int(year)}"
        # Convert the string to a datetime object
        pt_df.loc[idx, 'Datum'] = pd.to_datetime(date_str, format='%d-%m-%Y', errors='coerce')

# Drop 'Jaar', 'Maand', and 'Dag' columns if no longer needed
pt_df.drop(['Jaar', 'Maand', 'Dag'], axis=1, inplace=True)

# Convert 'Datum' to datetime
pt_df['Datum'] = pd.to_datetime(pt_df['Datum'], format='%d-%m-%Y', errors='coerce')
pt_df['Uur'] = pd.to_numeric(pt_df['Uur'], errors='coerce')

# Filter rows for the year 2020 and create a new DataFrame to avoid SettingWithCopyWarning
data_2020 = pt_df[pt_df['Datum'].dt.year == 2020].copy()

# Apply the function using loc to avoid SettingWithCopyWarning
data_2020['Datum'] = data_2020['Datum'].apply(adjust_year)
data_2020.dropna(subset=['Datum'], inplace=True)

# Handle references and renaming within data_2020 to avoid warnings
data_2020['Aantal_check_ins'] = data_2020['Referentie_vorig_jaar']

data_2020 = data_2020.drop(['Referentie_pre_COVID_19', 'Referentie_vorig_jaar', 
                            'Delta_actueel_pre_COVID_19', 'Delta_actueel_vorig_jaar'], axis=1)

# Drop the same columns from pt_df
pt_df = pt_df.drop(['Referentie_pre_COVID_19', 'Referentie_vorig_jaar', 'Delta_actueel_pre_COVID_19', 'Delta_actueel_vorig_jaar'], axis=1)

# Concatenate data_2020 back to pt_df
pt_df = pd.concat([pt_df, data_2020], ignore_index=True)
pt_df = pt_df.sort_values(by=['Datum', 'Uur'])
pt_df = pt_df.drop_duplicates()
data_2020.dropna(subset=['Datum'], inplace=True)

# Convert 'Datum' column to integer format YYYYMMDD
pt_df['Datum'] = pt_df['Datum'].dt.strftime('%Y%m%d').astype(int)

pt_df['Aantal_check_ins'] = np.where(pt_df['Datum'] < 20230000, 
                                     pt_df['Aantal_check_ins'] * 1000, 
                                     pt_df['Aantal_check_ins'])

In [73]:
# Add a new column for the previous hour, subtracting 1 only if HourOfDay is not 0
df['PrevHourOfDay'] = np.where(df['HourOfDay'] != 0, df['HourOfDay'] - 1, df['HourOfDay'])

# Now perform the merge
df = pd.merge(df, pt_df,
                     how='left',
                     left_on=['HourOfDay', 'DatumFileBeginInt'],
                     right_on=['Uur', 'Datum'],
                     suffixes=('', '_current'))

# Merge again for the previous hour
df = pd.merge(df, pt_df,
                     how='left',
                     left_on=['PrevHourOfDay', 'DatumFileBeginInt'],
                     right_on=['Uur', 'Datum'],
                     suffixes=('_current', '_previous'))

# Drop the intermediate column
df.drop('PrevHourOfDay', axis=1, inplace=True)


In [74]:
# Read the CSV file
free_day_df = pd.read_csv('data_freedays/freedays20192024.csv')

# Convert DatumFileBeginInt column to datetime
df['DatumFileBeginInt'] = pd.to_datetime(df['DatumFileBeginInt'], format='%Y%m%d')

# Convert Date column to datetime
free_day_df['Date'] = pd.to_datetime(free_day_df['Date'])

# Set index of free_day_df to 'Date' column
free_day_df.set_index('Date', inplace=True)

# Create a new column called 'freeday' based on comparison of dates
df['freeday'] = df['DatumFileBeginInt'].isin(free_day_df.index).astype(int)

In [75]:
df.to_csv('full_dataset.csv', index=False)